# Penguin Classification

*Can you train a machine learning model to pick out a penguin?*

In this notebook, you'll explore the basics of using the Apache Spark SQL and MLLib libraries in Azure Databricks to train and test a machine learning model. The scenario for this notebook is based on observations of penguins in Antartica, with the goal of trainign a machine learning model to predict the species of an observed penguin based on its location and body measurements.

> **Citation**: The penguins dataset used in the this exercise is a subset of data collected and made available by [Dr. Kristen
Gorman](https://www.uaf.edu/cfos/people/faculty/detail/kristen-gorman.php)
and the [Palmer Station, Antarctica LTER](https://pal.lternet.edu/), a
member of the [Long Term Ecological Research
Network](https://lternet.edu/).

## Ingest data

Run the following cell to ingest the data file you will use in this exercise. The data file will be saved in the DBFS storage for your Azure Databricks cluster.

In [0]:
%sh
rm -r /dbfs/data
mkdir /dbfs/data
wget -O /dbfs/data/penguins.csv https://raw.githubusercontent.com/MicrosoftLearning/dp-090-databricks-ml/master/data/penguins.csv

## Explore and clean up the data
  
Now that you've uploaded the data file, you can run the code in the following cell to load it into a dataframe and view it

In [0]:
df = spark.read.format("csv").option("header", "true").load("/data/penguins.csv")
display(df)

Island,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
Torgersen,39.1,18.7,181,3750,0
Torgersen,39.5,17.4,186,3800,0
Torgersen,40.3,18,195,3250,0
Torgersen,null,null,null,null,0
Torgersen,36.7,19.3,193,3450,0
Torgersen,39.3,20.6,190,3650,0
Torgersen,38.9,17.8,181,3625,0
Torgersen,39.2,19.6,195,4675,0
Torgersen,34.1,18.1,193,3475,0
Torgersen,42,20.2,190,4250,0


The code initiates the necessary *Spark Jobs* to load the data, and the output is a *pyspark.sql.dataframe.DataFrame* object named *df*. You'll see this information displayed directly under the code, and you can use the **&#9656;** toggle to expand the **df: pyspark.sql.dataframe.DataFrame** output and see details of the columns it contains and their data types. Since this data was loaded from a text file and contained some blank values, Spark has assigned a **string** data type to all of the columns.

The data itself consists of measurements of the following details of penguins that have been observed in Antartica:

- **Island**: The island in Antartica where the penguin was observed.
- **CulmenLength**: The length in mm of the penguin's culmen (bill).
- **CulmenDepth**: The depth in mm of the penguin's culmen.
- **FlipperLength**: The length in mm of the penguin's flipper.
- **BodyMass**: The body mass of the penguin in grams.
- **Species**: An integer value that represents the species of the penguin:
  - **0**: *Adelie*
  - **1**: *Gentoo*
  - **2**: *Chinstrap*
  
Our goal in this project is to use the observed charateristics of a pengun (its *features*) in order to predict its species (which in machine learning terminology, we call the *label*).
  
Note that some observations contain *null* or "missing" data values for some features. It's not uncommon for the raw source data you ingest to have issues like this, so typically the first stage in a machine learning project is to explore the data thoroughly and clean it up to make it more suitable for training a machine learning model.

A *dataframe* is a useful structure for performing this kind of data preparation work in Spark. For example, run the following cell to remove the rows with incomplete data by using the **dropna** method, and to apply appropriate data types to the data by using the **select** method with the **col** and **astype** functions.

> **Tip**: You can learn more about methods and functions you can use with dataframes in the [Spark SQL reference](https://spark.apache.org/docs/latest/sql-programming-guide.html).

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

data = df.dropna().select(col("Island").astype("string"),
                          col("CulmenLength").astype("float"),
                          col("CulmenDepth").astype("float"),
                          col("FlipperLength").astype("float"),
                          col("BodyMass").astype("float"),
                          col("Species").astype("int")
                          )
display(data)

Island,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species
Torgersen,39.1,18.7,181.0,3750.0,0
Torgersen,39.5,17.4,186.0,3800.0,0
Torgersen,40.3,18.0,195.0,3250.0,0
Torgersen,36.7,19.3,193.0,3450.0,0
Torgersen,39.3,20.6,190.0,3650.0,0
Torgersen,38.9,17.8,181.0,3625.0,0
Torgersen,39.2,19.6,195.0,4675.0,0
Torgersen,34.1,18.1,193.0,3475.0,0
Torgersen,42.0,20.2,190.0,4250.0,0
Torgersen,37.8,17.1,186.0,3300.0,0


Once again, you can toggle the details of the dataframe that is returned (this time named *data*) to verify that the data types have been applied, and you can review the data to verify that the rows containing incomplete data have been removed.

In a real project, you'd likely need to perform more exploration and data cleansing to fix (or remove) errors in the data, identify and remove outliers (non-tyically large or small values), or to balance the data so there's a reasonably equal number of rows for each label you're trying to predict.

## Split the data

For the purposes of this exercise, we'll assume that the data is now suitable cleaned and ready for us to use to train a machine learning model. The label we're going to try to predict is a specific category or *class* (the species of a penguin), so the type of machine learning model we need to train is a *classification* model. Classification (along with *regression*, which is used to predict a numeric value) is a form or *supervised* machine learning in which we use training data that includes known values for the label we want to predict. The process of training a model is really just fitting an algorithm to the data in order to calculate how the feature values correlate to the known label value. We can then apply the trained model to a new observation for which we know the only the feature values and have it predict the label value.

To ensure we can have confidence in our trained model, the typical approach is to train the model with only *some* of the data, and hold back some data with known label values that we can use to test the trained model and see how accurately its predictions are. To accomplish this goal, we'll split the full data set into two randomized subsets. We'll use 70% of the data for training, and hold back 30% for testing.

In [0]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 225  Testing Rows: 117


## Perform feature engineering

Having cleansed the raw data, a data scientists typically performs some additional work to prepare it for model training. This process is commonly known as *feature engineering*, and involves iteratively optimizing the features in the training dataset to produce the best possible model. The specific feature modifications required depends on the data and the desired model, but there are some common feature engineering tasks you should become familiar with.

### Encode categorical features

Machine learning algorithms are usually based on finding mathematical relationships between features and labels. That means that its usually best to define the features in your training data as *numeric* values. In some cases, you may have some features that are *categorical* rather than numeric, and which are expressed as strings - for example, the name of the island where the penguin observation occured in our dataset. However, most algorithms expect numeric features; so these string-based categorical values need to be *encoded* as numbers. In this case, we'll use a **StringIndexer** from the **Spark MLLib** library to encode the island name as a numeric value by assigning a unique interger index for each discrete island name.

Run the following cell to see how the **Island** categorical column values will be encoded in our training data.

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Island", outputCol="IslandIdx")
indexedData = indexer.fit(train).transform(train).drop("Island")
display(indexedData)

CulmenLength,CulmenDepth,FlipperLength,BodyMass,Species,IslandIdx
34.5,18.1,187.0,2900.0,0,0.0
35.0,17.9,190.0,3450.0,0,0.0
35.0,17.9,192.0,3725.0,0,0.0
35.9,19.2,189.0,3800.0,0,0.0
36.4,17.1,184.0,2850.0,0,0.0
36.5,16.6,181.0,2850.0,0,0.0
37.6,17.0,185.0,3600.0,0,0.0
37.6,19.1,194.0,3750.0,0,0.0
37.9,18.6,172.0,3150.0,0,0.0
37.9,18.6,193.0,2925.0,0,0.0


In the results, you should see that instead of an island name, each row now has an **IslandIdx** column with an integer value representing the island on which the observation was recorded.

### Normalize (scale) numeric features

Now let's turn our attention to the numeric values in our data. These values (**CulmenLength**, **CulmenDepth**, **FlipperLength**, and **BodyMass**) all represent measurements of one sort or another, but they're in different scales. When training a model, the units of measurement aren't as important as the relative differences across different observations, and features that are represented by larger numbers can often dominate the model training algorithm - skewing the importance of the feature when calculating a prediction. To mitigate this, it's common to *normalize* the numeric feature values so they're all on the same relative scale (for example, a decimal value between 0.0 and 1.0).

The code we'll use to do this is a little more involved than the categorical encoding we did previously. We need to scale multiple column values at the same time, so the technique we use is to create a single column containing a *vector* (essentially an array) of all the numeric features, and then apply a scaler to produce a new vector column with the equivalent normalized values.

Run the cell below to normalize the numeric features and see a comparison of the pre-normlized and normalized vector columns.

In [0]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

#Create a vector column containing all numeric features
numericFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
numericColVector = VectorAssembler(inputCols=numericFeatures, outputCol="numericFeatures")
vectorizedData = numericColVector.transform(indexedData)

#Use a MinMax scaler to normalize the numeric values in the vector
minMax = MinMaxScaler(inputCol = numericColVector.getOutputCol(), outputCol="normalizedFeatures")
scaledData = minMax.fit(vectorizedData).transform(vectorizedData)

# Display the data with numeric feature vectors (before and after scaling)
compareNumerics = scaledData.select("numericFeatures", "normalizedFeatures")
display(compareNumerics)

numericFeatures,normalizedFeatures
"Map(vectorType -> dense, length -> 4, values -> List(34.5, 18.100000381469727, 187.0, 2900.0))","Map(vectorType -> dense, length -> 4, values -> List(0.08727278275923295, 0.5952381222696814, 0.2542372881355932, 0.05555555555555555))"
"Map(vectorType -> dense, length -> 4, values -> List(35.0, 17.899999618530273, 190.0, 3450.0))","Map(vectorType -> dense, length -> 4, values -> List(0.10545460094105114, 0.5714285065527647, 0.3050847457627119, 0.20833333333333334))"
"Map(vectorType -> dense, length -> 4, values -> List(35.0, 17.899999618530273, 192.0, 3725.0))","Map(vectorType -> dense, length -> 4, values -> List(0.10545460094105114, 0.5714285065527647, 0.3389830508474576, 0.2847222222222222))"
"Map(vectorType -> dense, length -> 4, values -> List(35.900001525878906, 19.200000762939453, 189.0, 3800.0))","Map(vectorType -> dense, length -> 4, values -> List(0.13818192915482955, 0.726190554582076, 0.288135593220339, 0.3055555555555556))"
"Map(vectorType -> dense, length -> 4, values -> List(36.400001525878906, 17.100000381469727, 184.0, 2850.0))","Map(vectorType -> dense, length -> 4, values -> List(0.1563637473366477, 0.47619049781574513, 0.2033898305084746, 0.041666666666666664))"
"Map(vectorType -> dense, length -> 4, values -> List(36.5, 16.600000381469727, 181.0, 2850.0))","Map(vectorType -> dense, length -> 4, values -> List(0.16000005548650567, 0.416666685588777, 0.15254237288135594, 0.041666666666666664))"
"Map(vectorType -> dense, length -> 4, values -> List(37.599998474121094, 17.0, 185.0, 3600.0))","Map(vectorType -> dense, length -> 4, values -> List(0.19999999999999998, 0.46428568995728675, 0.22033898305084745, 0.25))"
"Map(vectorType -> dense, length -> 4, values -> List(37.599998474121094, 19.100000381469727, 194.0, 3750.0))","Map(vectorType -> dense, length -> 4, values -> List(0.19999999999999998, 0.7142857467236177, 0.3728813559322034, 0.2916666666666667))"
"Map(vectorType -> dense, length -> 4, values -> List(37.900001525878906, 18.600000381469727, 172.0, 3150.0))","Map(vectorType -> dense, length -> 4, values -> List(0.21090920188210227, 0.6547619344966495, 0.0, 0.125))"
"Map(vectorType -> dense, length -> 4, values -> List(37.900001525878906, 18.600000381469727, 193.0, 2925.0))","Map(vectorType -> dense, length -> 4, values -> List(0.21090920188210227, 0.6547619344966495, 0.3559322033898305, 0.0625))"


The **numericFeatures** column in the results contains a vector for each row. The vector includes four unscaled numeric values (the original measurements of the penguin). You can use the **&#9656;** toggle to see the discrete values more clearly.

The **normalizedFeatures** column also contains a vector for each penguin observation, but this time the values in the vector are normalized to a relative scale based on the minimum and maximum values for each measurement.

### Prepare features and labels for training

Now, let's bring everything together and create a single column containing all of the feaures (the encoded categorical island name and the normalized penguin measurements), and another column containing the class label we want to train a model to predict (the penguin species).

In [0]:
featVect = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="featuresVector")
preppedData = featVect.transform(scaledData)[col("featuresVector").alias("features"), col("Species").alias("label")]
display(preppedData)

features,label
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.08727278275923295, 0.5952381222696814, 0.2542372881355932, 0.05555555555555555))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.10545460094105114, 0.5714285065527647, 0.3050847457627119, 0.20833333333333334))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.10545460094105114, 0.5714285065527647, 0.3389830508474576, 0.2847222222222222))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.13818192915482955, 0.726190554582076, 0.288135593220339, 0.3055555555555556))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.1563637473366477, 0.47619049781574513, 0.2033898305084746, 0.041666666666666664))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.16000005548650567, 0.416666685588777, 0.15254237288135594, 0.041666666666666664))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19999999999999998, 0.46428568995728675, 0.22033898305084745, 0.25))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19999999999999998, 0.7142857467236177, 0.3728813559322034, 0.2916666666666667))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.21090920188210227, 0.6547619344966495, 0.0, 0.125))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.21090920188210227, 0.6547619344966495, 0.3559322033898305, 0.0625))",0


The **features** vector contains five values (the encoded island and the normalized culmen length, culmen depth, flipper length, and body mass). The label contains a sinple integer code that indicates the class of penguin species.

## Train a machine learning model

Now that the training data is prepared, you can use it to train a model. Models are trained using an *algorithm* that tries to establish a relationship between the features and labels. Since in this case you want to train a model that predicts a category of *class* , you need to use a *classification* algorthm. There are many algorithms for classification - let's start with a well-establish one: logistic regression, which iteratively attempts to find the optimal coefficients that can be applied to the features data in a logistic calculation that predicts the probability for each class label value. To train the model, you will fit the logistic regression algorithm to the training data.

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(preppedData)
print ("Model trained!")

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Model trained!


Most algorithms support parameters that give you some control over the way the model is trained. In this case, the logistic regression algorithm require you to identify the column containing the features vector and the column containing the known label; and also enables you to specify the maximum number of iterations performed to find optimal coeficients for the logistic calculation, and a regularization parameter that is used to prevent the model from *overfitting* (in other words, establishing a logistic calculation that works well with the training data, but which doesn't generalize well when applied to new data).

## Test the model

Now that you have a trained model, you can test it with the data you held back. Before you can do this, you need to perform the same feature engineering transformations to the test data as you applied to the training data (in this case, encode the island name and normalize the measurements).

In [0]:
indexedTestData = indexer.fit(test).transform(test).drop("Island")
vectorizedTestData = numericColVector.transform(indexedTestData)
scaledTestData = minMax.fit(vectorizedTestData).transform(vectorizedTestData)
preppedTestData = featVect.transform(scaledTestData)[col("featuresVector").alias("features"), col("Species").alias("label")]
display(preppedTestData)

features,label
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.059360691620242906, 0.7124998912215362, 0.23214285714285712, 0.29508196721311475))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.0684931459126828, 0.3750000745057971, 0.375, 0.14754098360655737))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.0776256002051227, 0.46249992102385507, 0.19642857142857142, 0.08196721311475409))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.16894979475536645, 0.3499999821186087, 0.1607142857142857, 0.05737704918032787))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.16894979475536645, 0.6875000372528985, 0.10714285714285714, 0.22950819672131148))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.17351593480804758, 0.6374998524785218, 0.0, 0.16393442622950818))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.17351593480804758, 0.849999922513971, 0.2857142857142857, 0.4426229508196721))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19178084339292736, 0.612499998509884, 0.19642857142857142, 0.3442622950819672))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19178084339292736, 0.849999922513971, 0.2857142857142857, 0.32786885245901637))",0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.21004557779072955, 0.5000000596046377, 0.4464285714285714, 0.2786885245901639))",0


Now you have test data in the appropriate format for the trained model. You can use the feature vectors in the test data to generate predicted class labels, and compare them to the known (true) labels to evaluate how well the model performs.

In [0]:
prediction = model.transform(preppedTestData)
predicted = prediction.select("features", "probability", col("prediction").astype("Int"), col("label").alias("trueLabel"))
display(predicted)

features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.059360691620242906, 0.7124998912215362, 0.23214285714285712, 0.29508196721311475))","Map(vectorType -> dense, length -> 3, values -> List(0.870844629116958, 0.05390328099421903, 0.07525208988882298))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.0684931459126828, 0.3750000745057971, 0.375, 0.14754098360655737))","Map(vectorType -> dense, length -> 3, values -> List(0.7738748390841244, 0.13425329592912003, 0.09187186498675563))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.0776256002051227, 0.46249992102385507, 0.19642857142857142, 0.08196721311475409))","Map(vectorType -> dense, length -> 3, values -> List(0.8377676318052165, 0.06699532370420212, 0.0952370444905813))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.16894979475536645, 0.3499999821186087, 0.1607142857142857, 0.05737704918032787))","Map(vectorType -> dense, length -> 3, values -> List(0.7658304796100598, 0.09783725570833202, 0.13633226468160806))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.16894979475536645, 0.6875000372528985, 0.10714285714285714, 0.22950819672131148))","Map(vectorType -> dense, length -> 3, values -> List(0.8328737246064294, 0.05106915477176535, 0.11605712062180523))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.17351593480804758, 0.6374998524785218, 0.0, 0.16393442622950818))","Map(vectorType -> dense, length -> 3, values -> List(0.8398118921591838, 0.04065675125271028, 0.11953135658810597))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.17351593480804758, 0.849999922513971, 0.2857142857142857, 0.4426229508196721))","Map(vectorType -> dense, length -> 3, values -> List(0.8178487311103401, 0.07561424284252775, 0.10653702604713217))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19178084339292736, 0.612499998509884, 0.19642857142857142, 0.3442622950819672))","Map(vectorType -> dense, length -> 3, values -> List(0.7807481489415493, 0.09779762897367236, 0.12145422208477828))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.19178084339292736, 0.849999922513971, 0.2857142857142857, 0.32786885245901637))","Map(vectorType -> dense, length -> 3, values -> List(0.8129117047388373, 0.0643453057925973, 0.12274298946856528))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.21004557779072955, 0.5000000596046377, 0.4464285714285714, 0.2786885245901639))","Map(vectorType -> dense, length -> 3, values -> List(0.6648059012416997, 0.19743055209403476, 0.13776354666426555))",0,0


The results of the previous cell include the following columns:

- **features**: The prepared features data from the test dataset.
- **probability**: The probability calculated by the model for each class. This consists of a vector containing three probability values (because there are three classes) which add up to a total of 1.0 (its assumed that there's a 100% probability that the penguin belongs to *one* of the three species classes).
- **prediction**: The predicted class label (the one with the highest probability).
- **trueLabel**: The actual known label value from the test data.

Tp evaluate the effectiveness of the model, you could simply compare the predicted and true labels in these results. However, you can get more meaningful metrics by using a model evaluator - in this case, a multiclass (because there are multiple possible class labels) classification evaluator.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

# Simple accuracy
accuracy = evaluator.evaluate(prediction, {evaluator.metricName:"accuracy"})
print("Accuracy:", accuracy)

# Class metrics
labels = [0,1,2]
print("\nIndividual class metrics:")
for label in sorted(labels):
    print ("Class %s" % (label))

    # Precision
    precision = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                                    evaluator.metricName:"precisionByLabel"})
    print("\tPrecision:", precision)

    # Recall
    recall = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                                 evaluator.metricName:"recallByLabel"})
    print("\tRecall:", recall)

    # F1 score
    f1 = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                             evaluator.metricName:"fMeasureByLabel"})
    print("\tF1 Score:", f1)

# Weighed (overall) metrics
overallPrecision = evaluator.evaluate(prediction, {evaluator.metricName:"weightedPrecision"})
print("Overall Precision:", overallPrecision)
overallRecall = evaluator.evaluate(prediction, {evaluator.metricName:"weightedRecall"})
print("Overall Recall:", overallRecall)
overallF1 = evaluator.evaluate(prediction, {evaluator.metricName:"weightedFMeasure"})
print("Overall F1 Score:", overallF1)

Accuracy: 0.9401709401709402

Individual class metrics:
Class 0
	Precision: 0.875
	Recall: 1.0
	F1 Score: 0.9333333333333333
Class 1
	Precision: 1.0
	Recall: 1.0
	F1 Score: 1.0
Class 2
	Precision: 1.0
	Recall: 0.6956521739130435
	F1 Score: 0.8205128205128205
Overall Precision: 0.9476495726495726
Overall Recall: 0.9401709401709402
Overall F1 Score: 0.9367959675651983


The evaluation metrics that are calculated for multiclass classification include:

- **Accuracy**: The proportion of overall predictions that were correct.
- Per-class metrics:
  - **Precision**: The proportion of predictions of this class that were correct.
  - **Recall**: The proportion of actual instances of this class that were correctly predicted.
  - **F1 score**: A combined metric for precision and recall
- Combined (weighted) precision, recall, and F1 metrics for all classes.

> **Note**: It may initially seem like the overall accuracy metric provids the best way to evaluare a model's predictive performance. However, consider this. Suppose Gentoo penguins make up 95% of the penguin population in your study location. A model that always predicts the label **1** (the class for Gentoo) will have an accuracy of 0.95. That doesn't mean it's a great model for predicting a penguin species based on the features! That's why data scientists tend to explore additional metrics to get a better understanding of how well a classification model predicts for each possible class label.

## Use a pipeline

You trained your model by performing the required feature engineering steps and then fitting an algorithm to the data. To use the model with some test data to generate predictions (referred to as *inferencing*), you had to apply the same feature engineering steps to the test data. A more efficient way to build and use models is to encapsulate the transformers used to prepare the data and the model used to train it in a *pipeline*.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import LogisticRegression

catFeature = "Island"
numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]

# Define the feature engineering and model steps
catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
algo = LogisticRegression(labelCol="Species", featuresCol="Features", maxIter=10, regParam=0.3)

# Chain the steps as stages in a pipeline
pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, algo])

# Use the pipeline to prepare data and fit the model algorithm
model = pipeline.fit(train)
print ("Model trained!")

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Model trained!


Since the feature engineering steps are now encapsulated in the model trained by the pipeline, you can use the model with the test data without needing to apply each transformation (they'll be applied automatically by the model).

In [0]:
prediction = model.transform(test)
predicted = prediction.select("Features", "probability", col("prediction").astype("Int"), col("Species").alias("trueLabel"))
display(predicted)

Features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.1163636641068892, 0.6904761310067009, 0.2542372881355932, 0.3055555555555556))","Map(vectorType -> dense, length -> 3, values -> List(0.8344375457611268, 0.07103975426742509, 0.09452269997144795))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.12363641912286931, 0.3690476812202672, 0.3898305084745763, 0.18055555555555555))","Map(vectorType -> dense, length -> 3, values -> List(0.7228273342125653, 0.16807364305984868, 0.10909902272758609))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.13090917413884942, 0.4523808820988284, 0.22033898305084745, 0.125))","Map(vectorType -> dense, length -> 3, values -> List(0.796384951276815, 0.08943031050036664, 0.11418473822281838))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.20363644686612214, 0.34523806550335046, 0.1864406779661017, 0.10416666666666667))","Map(vectorType -> dense, length -> 3, values -> List(0.7277020889037037, 0.1225458280329951, 0.1497520830633013))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.20363644686612214, 0.6666667423551079, 0.13559322033898305, 0.25))","Map(vectorType -> dense, length -> 3, values -> List(0.8028119623532536, 0.06499024923825286, 0.13219778840849358))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.2072727550159801, 0.6190475109212744, 0.03389830508474576, 0.19444444444444445))","Map(vectorType -> dense, length -> 3, values -> List(0.8115274016074162, 0.05305935907275653, 0.13541323931982727))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.2072727550159801, 0.8214285633190956, 0.3050847457627119, 0.4305555555555556))","Map(vectorType -> dense, length -> 3, values -> List(0.7872619786753925, 0.0900348079535856, 0.12270321337102197))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.22181826504794033, 0.5952381222696814, 0.22033898305084745, 0.3472222222222222))","Map(vectorType -> dense, length -> 3, values -> List(0.7487592028655302, 0.11559549878103571, 0.13564529835343409))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.22181826504794033, 0.8214285633190956, 0.3050847457627119, 0.3333333333333333))","Map(vectorType -> dense, length -> 3, values -> List(0.7837895167437481, 0.07834935574356695, 0.13786112751268492))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.23636363636363636, 0.4880953056742035, 0.4576271186440678, 0.2916666666666667))","Map(vectorType -> dense, length -> 3, values -> List(0.6305745741272093, 0.22187631897315585, 0.14754910689963496))",0,0


## Try a different algorithm

So far you've trained a classification model by using the logistic regression algorithm. Let's change that stage in the pipeline to try a different algorithm.

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import DecisionTreeClassifier

catFeature = "Island"
numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]

# Define the feature engineering and model steps
# catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
# numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
# numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
# featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
algo = DecisionTreeClassifier(labelCol="Species", featuresCol="Features", maxDepth=10)

# Chain the steps as stages in a pipeline
pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, algo])

# Use the pipeline to prepare data and fit the model algorithm
model = pipeline.fit(train)
print ("Model trained!")

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Model trained!


This time, the pipeline includes the same feature preparation stages as before but uses a *Decision Tree* algorithm to train the model.

Once again, you can use the trained model to inference predictions from the test data.

In [0]:
prediction = model.transform(test)
predicted = prediction.select("Features", "probability", col("prediction").astype("Int"), col("Species").alias("trueLabel"))
display(predicted)

Features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.1163636641068892, 0.6904761310067009, 0.2542372881355932, 0.3055555555555556))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.12363641912286931, 0.3690476812202672, 0.3898305084745763, 0.18055555555555555))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.13090917413884942, 0.4523808820988284, 0.22033898305084745, 0.125))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.20363644686612214, 0.34523806550335046, 0.1864406779661017, 0.10416666666666667))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.20363644686612214, 0.6666667423551079, 0.13559322033898305, 0.25))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.2072727550159801, 0.6190475109212744, 0.03389830508474576, 0.19444444444444445))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.2072727550159801, 0.8214285633190956, 0.3050847457627119, 0.4305555555555556))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.22181826504794033, 0.5952381222696814, 0.22033898305084745, 0.3472222222222222))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.22181826504794033, 0.8214285633190956, 0.3050847457627119, 0.3333333333333333))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0
"Map(vectorType -> dense, length -> 5, values -> List(0.0, 0.23636363636363636, 0.4880953056742035, 0.4576271186440678, 0.2916666666666667))","Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.0, 0.0))",0,0


Now let's evaluate the performance of the new model.

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction")

# Simple accuracy
accuracy = evaluator.evaluate(prediction, {evaluator.metricName:"accuracy"})
print("Accuracy:", accuracy)

# Class metrics
labels = [0,1,2]
print("\nIndividual class metrics:")
for label in sorted(labels):
    print ("Class %s" % (label))

    # Precision
    precision = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                                    evaluator.metricName:"precisionByLabel"})
    print("\tPrecision:", precision)

    # Recall
    recall = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                                 evaluator.metricName:"recallByLabel"})
    print("\tRecall:", recall)

    # F1 score
    f1 = evaluator.evaluate(prediction, {evaluator.metricLabel:label,
                                             evaluator.metricName:"fMeasureByLabel"})
    print("\tF1 Score:", f1)

# Weighed (overall) metrics
overallPrecision = evaluator.evaluate(prediction, {evaluator.metricName:"weightedPrecision"})
print("Overall Precision:", overallPrecision)
overallRecall = evaluator.evaluate(prediction, {evaluator.metricName:"weightedRecall"})
print("Overall Recall:", overallRecall)
overallF1 = evaluator.evaluate(prediction, {evaluator.metricName:"weightedFMeasure"})
print("Overall F1 Score:", overallF1)


Accuracy: 0.9829059829059829

Individual class metrics:
Class 0
	Precision: 0.9795918367346939
	Recall: 0.9795918367346939
	F1 Score: 0.9795918367346939
Class 1
	Precision: 1.0
	Recall: 1.0
	F1 Score: 1.0
Class 2
	Precision: 0.9565217391304348
	Recall: 0.9565217391304348
	F1 Score: 0.9565217391304348
Overall Precision: 0.9829059829059829
Overall Recall: 0.9829059829059829
Overall F1 Score: 0.9829059829059829


It seems that the decision tree algorithm has resulted in model with slightly better predictive performance.

## Save the model

In reality, you'd iteratively try training the model with different algorithms (and parameters) to find the best model for your data. For now, we'll stick with the decision trees model we've trained. Let's save it so we can use it later with some new penguin observations.

In [0]:
model.save("/models/penguin.model")

## Use the model for inferencing

Now, when you've been out and spotted a new penguin, you can load the model and use it to predict the penguin's species based on your measurements of its features. Using a model to generate predictions from new data is called *inferencing*.

In [0]:
from pyspark.ml.pipeline import PipelineModel

persistedModel = PipelineModel.load("/models/penguin.model")

newData = spark.createDataFrame ([{"Island": "Biscoe",
                                  "CulmenLength": 47.6,
                                  "CulmenDepth": 14.5,
                                  "FlipperLength": 215,
                                  "BodyMass": 5400}])


predictions = persistedModel.transform(newData)
display(predictions.select("Island", "CulmenDepth", "CulmenLength", "FlipperLength", "BodyMass", col("prediction").alias("PredictedSpecies")))

Island,CulmenDepth,CulmenLength,FlipperLength,BodyMass,PredictedSpecies
Biscoe,14.5,47.6,215,5400,1.0


In this notebook, you've explored the basics of preparing data and training machine learning models using MLLib in Apache Spark.

For more information see the [Spark MLLib documentation](https://spark.apache.org/docs/latest/ml-guide.html).